In [30]:
#IMPORTS
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
import torch
from urllib import request
import random
# device = torch.device("cpu")

In [21]:
# # prepare logger
# logging.basicConfig(level=logging.INFO)

# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # check gpu
# cuda_available = torch.cuda.is_available()

# print('Cuda available? ',cuda_available)
# if cuda_available:
#   import tensorflow as tf
#   # Get the GPU device name.
#   device_name = tf.test.gpu_device_name()
#   # The device name should look like the following:
#   if device_name == '/device:GPU:0':
#       print('Found GPU at: {}'.format(device_name))
#   else:
#       raise SystemError('GPU device not found')

Cuda available?  False


In [22]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [23]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('.', '.')
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [33]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)
data=dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [34]:

rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  item = {
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  }
  rows.append(item)

In [35]:
trdf1 = pd.DataFrame(rows)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [37]:
test_rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  test_rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
  #TODO: keyword?Country?length?

In [39]:
len(rows)

2094

In [47]:
tedf1 = pd.DataFrame(test_rows)
tedf1

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [82]:
tedf1 = tedf1.sample(frac = 1) # shuffle data
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])
# tedf1.par_id
train_data = training_set1
test_data = tedf1
train_data_text_list = train_data.text.to_list()
train_data_label_list = train_data.label.to_list()
test_data_text_list = test_data.text.to_list()
test_data_label_list = test_data.label.to_list()

In [77]:
from sklearn.metrics import accuracy_score, f1_score
def calc_scores_and_print_miscalculations(true_lables, pred_labels, unseen_data, n=10):
    '''prints accuracy, f1 score and first N miscalculations'''
    calc_scores(true_lables, pred_labels)
    # Print predicted labels for unseen test data
    print("\nMisclassifications:")
    c = 0
    for text, true_label, pred_label in zip(unseen_data, true_lables, pred_labels):
        if true_label != pred_label:
            print(f"Pred: {pred_label} Ac: {true_label} Text: {text}")
            c += 1
            if c == n:
                break

def calc_scores(true_lables, pred_labels):
    # Evaluate the model
    accuracy = accuracy_score(true_lables, pred_labels)
    print("Accuracy:", accuracy)

    f1 = f1_score(true_lables, pred_labels)
    print("F1 Score:", f1)

## Benchmark - Bag of words (no augmentation)

In [80]:
# Title
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# Example dataset (X: text data, y: sentiment labels)
X = train_data_text_list
y = train_data_label_list  # 1: Positive, 0: Negative (Binary sentiment labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create BoW vectors
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train a logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train_bow, y_train)

# Predict sentiment labels for the test set
y_pred = classifier.predict(X_test_bow)

# Evaluate the model
calc_scores(y_test, y_pred)

# Unseen data
unseen_data = test_data_text_list
true_labels = test_data_label_list

# Vectorize the unseen test data using the same CountVectorizer instance
X_unseen_bow = vectorizer.transform(unseen_data)

# Predict sentiment labels for the unseen test data
y_unseen_pred = classifier.predict(X_unseen_bow)

# Calculate accuracy
calc_scores_and_print_miscalculations(true_labels, y_unseen_pred, unseen_data)

Accuracy: 0.906865671641791
F1 Score: 0.3217391304347826
Accuracy: 0.894937917860554
F1 Score: 0.20863309352517986

Misclassifications:
Pred: 0 Ac: 1 Text: Speaking in Garissa on Sunday when he hosted officials from Qatar Charity who are financing a programme to assist two orphanages in the county , Korane said vulnerable children stand equal chances of becoming useful members of society if given good upbringing and education .
Pred: 0 Ac: 1 Text: 18 December should serve as a time when we look with compassion at the fate of migrants , refugees and the internally displaced . It is especially a time when we must plan and increase resources for creative action .
Pred: 0 Ac: 1 Text: Fasting bridges the gap between rich and poor , sustained and impoverished , fulfilled and needy . This experience should then inspire compassion and mercy , which is manifest by generosity of wealth and time to help those in need .
Pred: 0 Ac: 1 Text: The World Health Organization did not give a reason for th

## Benchmark model - SVM - TF-IDF feature representation

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# Example dataset (X: text data, y: sentiment labels)
X = train_data_text_list
y = train_data_label_list # 1: Positive, 0: Negative (Binary sentiment labels)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment labels for the test set
y_pred = svm_classifier.predict(X_test_tfidf)

calc_scores(y_test, y_pred)

# Example of unseen test data
unseen_data = test_data_text_list
true_labels = test_data_label_list # True sentiment labels for the unseen test data

# Convert the unseen test data into TF-IDF vectors
X_unseen_tfidf = tfidf_vectorizer.transform(unseen_data)

# Predict sentiment labels for the unseen test data
y_unseen_pred = svm_classifier.predict(X_unseen_tfidf)

# Calculate accuracy for unseen test data
calc_scores_and_print_miscalculations(true_labels, y_unseen_pred, unseen_data)



Accuracy: 0.7610062893081762
F1 Score: 0.6174496644295302
Accuracy: 0.8414517669531996
F1 Score: 0.336

Misclassifications:
Pred: 0 Ac: 1 Text: TurkIt 's heartening to see that measures are being taken in Khyber Pakhtunkhwa ( KP ) to empower women and give them work opportunities . You ! takes a look ...
Pred: 0 Ac: 1 Text: """ We are taking a community that is fairly vulnerable and bringing them to a level where they are empowered . "" <h> VICTORIA KIOKO"
Pred: 1 Ac: 0 Text: The video shows her trying to read news that was just received from the Associated Press -- that there are so-called tender age shelters , where immigrant babies and young children are being placed after they 're separated from their parents at the U.S. border .
Pred: 1 Ac: 0 Text: Having gone through a lot as parents , one thing that the foundation seeks to champion is for the government to establish a hospice facility to lessen the burden on women who normally take care of family members whose medical conditions

In [62]:
test_data.text

0       We also know that they can benefit by receivin...
1       Pope Francis washed and kissed the feet of Mus...
2       Many refugees do n't want to be resettled anyw...
3       "Budding chefs , like "" Fred "" , "" Winston ...
4       "In a 90-degree view of his constituency , one...
                              ...                        
2089    The sad spectacle , which occurred on Saturday...
2090    """ The Pakistani police came to our house and...
2091    "When Marie O'Donoghue went looking for a spec...
2092    "Sri Lankan norms and culture inhibit women fr...
2093    He added that the AFP will continue to bank on...
Name: text, Length: 2094, dtype: object